<a href="https://colab.research.google.com/github/lucasaaz/Implementa-o-de-Assistente-Conversacional-Baseado-em-LLM/blob/main/AS05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comando Git

In [ ]:
# Instale o Git (caso não esteja instalado)
!apt-get install git -y

# Configure seu usuário (substitua com seus dados)
!git config --global user.name "Lucas"
!git config --global user.email "lucas.augustoaz@gmail.com"

In [ ]:
# Substitua pela URL do SEU repositório criado no GitHub
!git clone https://github.com/lucasaaz/Implementa-o-de-Assistente-Conversacional-Baseado-em-LLM.git

# Acesse a pasta do repositório
%cd nome-do-repositorio

In [ ]:
# Copie o notebook atual para o repositório
!cp /content/Seu_Notebook.ipynb /content/Implementa-o-de-Assistente-Conversacional-Baseado-em-LLM/

# Copie outros arquivos necessários (requirements.txt, etc)
!cp /content/requirements.txt /Implementa-o-de-Assistente-Conversacional-Baseado-em-LLM/

In [ ]:
# Adicione todos os arquivos
!git add .

# Commit das alterações
!git commit -m "Versão inicial do projeto - Publicação do Colab"

# Faça o push para o GitHub (insira seu token quando pedido)
# Gere um token em: GitHub > Settings > Developer Settings > Personal Access Tokens
!git push https://github.com/lucasaaz/Implementa-o-de-Assistente-Conversacional-Baseado-em-LLM.git main

# **Implementação de Assistente Conversacional Baseado em LLM**

In [ ]:
# %% [markdown]
# # Assistente Conversacional Baseado em LLM - AS05

# %%
# Instalação de dependências
!pip install -q langchain==0.1.4 openai==1.12.0 faiss-cpu==1.7.4 pypdf==3.17.4 tiktoken==0.5.2 python-dotenv==1.0.0 gradio==4.44.1 --upgrade
!pip install -q pdf2image pytesseract pillow
!sudo apt install -y tesseract-ocr
!sudo apt install -y libtesseract-dev

# %%
import os
import logging
import tempfile
from dotenv import load_dotenv
from pdf2image import convert_from_path
import pytesseract
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
import gradio as gr

# Configuração de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# %%
# Configuração
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or input("Digite sua OpenAI API Key: ")

# %%
def super_safe_string(text, default=""):
    """Versão ultra-protegida para conversão de strings"""
    try:
        if text is None:
            return default
        if isinstance(text, str):
            return text
        return str(text) if text else default
    except:
        return default

class PDFAssistant:
    def __init__(self):
        self.vectorstore = None
        self.qa_chain = None
        self.embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
        self.llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)
        logger.info("Assistente inicializado com sucesso")

    def extract_text_with_fallback(self, pdf_path):
        """Extrai texto com múltiplos fallbacks"""
        try:
            # Tenta extração normal primeiro
            loader = PyPDFLoader(pdf_path)
            docs = loader.load()
            if docs and super_safe_string(docs[0].page_content):
                return docs

            # Fallback para OCR se necessário
            logger.info(f"Tentando OCR para {pdf_path}")
            images = convert_from_path(pdf_path)
            ocr_docs = []
            for i, img in enumerate(images):
                text = pytesseract.image_to_string(img)
                if text.strip():
                    ocr_docs.append(Document(
                        page_content=text,
                        metadata={"source": pdf_path, "page": i+1}
                    ))
            return ocr_docs if ocr_docs else [Document(page_content="", metadata={"source": pdf_path})]

        except Exception as e:
            logger.error(f"Erro na extração: {str(e)}")
            return [Document(page_content="", metadata={"source": pdf_path})]

    def load_and_process_pdfs(self, pdf_files):
        """Processamento totalmente seguro de PDFs"""
        if not pdf_files:
            return "⚠️ Nenhum arquivo PDF foi enviado"

        documents = []
        for pdf_file in pdf_files:
            try:
                with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                    tmp_path = tmp_file.name
                    tmp_file.write(pdf_file.read())

                docs = self.extract_text_with_fallback(tmp_path)
                valid_docs = [doc for doc in docs if super_safe_string(doc.page_content).strip()]
                documents.extend(valid_docs)
                os.remove(tmp_path)

            except Exception as e:
                logger.error(f"Erro processando {pdf_file.name}: {str(e)}")
                if os.path.exists(tmp_path):
                    os.remove(tmp_path)
                continue

        if not documents:
            return "⚠️ Não foi possível extrair texto válido dos PDFs"

        try:
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200,
                length_function=len
            )
            chunks = text_splitter.split_documents(documents)

            self.vectorstore = FAISS.from_documents(chunks, self.embeddings)
            self.qa_chain = RetrievalQA.from_chain_type(
                self.llm,
                retriever=self.vectorstore.as_retriever(
                    search_type="mmr",
                    search_kwargs={"k": 4}
                ),
                return_source_documents=True
            )
            return f"✅ {len(chunks)} chunks processados com sucesso"

        except Exception as e:
            logger.error(f"Erro no processamento final: {str(e)}")
            return f"❌ Erro no processamento: {str(e)}"

    def ask_question(self, question):
        """Método ultra-protegido para perguntas"""
        try:
            safe_question = super_safe_string(question).strip()
            if not safe_question:
                return "⚠️ Pergunta inválida ou vazia"

            if not self.qa_chain:
                return "ℹ️ Documentos não carregados. Envie PDFs primeiro."

            result = self.qa_chain({"query": safe_question}) or {}

            answer = super_safe_string(result.get("result"), "Resposta não disponível")

            sources = []
            for doc in result.get("source_documents", []):
                try:
                    source = super_safe_string(doc.metadata.get("source"), "Fonte desconhecida")
                    page = super_safe_string(doc.metadata.get("page"))
                    if page:
                        source += f" (página {page})"
                    sources.append(source)
                except:
                    continue

            seen = set()
            unique_sources = [x for x in sources if not (x in seen or seen.add(x))]

            return f"Resposta: {answer}\n\nFontes:\n" + "\n".join(f"- {src}" for src in unique_sources)

        except Exception as e:
            logger.error(f"ERRO CRÍTICO: {str(e)}", exc_info=True)
            return "❌ Erro interno. Tente novamente ou recarregue os documentos."

# %%
assistant = PDFAssistant()

# %%
css = """
footer {visibility: hidden}
.gr-box {border: 1px solid #e2e2e2; border-radius: 8px;}
.gr-button-primary {background: #4f46e5; color: white;}
"""

def create_response(question, chat_history):
    try:
        response = assistant.ask_question(question)
        return "", chat_history + [(question, response)]
    except Exception as e:
        logger.error(f"Erro na interface: {str(e)}")
        return "", chat_history + [(question, "❌ Erro no sistema. Tente novamente.")]

with gr.Blocks(title="Assistente de PDF", theme="soft", css=css) as demo:
    gr.Markdown("""# 📄 Assistente de PDF Inteligente""")

    with gr.Tab("📤 Carregar PDFs"):
        gr.Markdown("Envie seus documentos para análise")
        files = gr.File(file_types=[".pdf"], file_count="multiple")
        upload_btn = gr.Button("Processar", variant="primary")
        status = gr.Textbox(label="Status", interactive=False)

    with gr.Tab("💬 Conversar"):
        chatbot = gr.Chatbot(height=400)
        msg = gr.Textbox(placeholder="Digite sua pergunta...")
        send_btn = gr.Button("Enviar", variant="primary")
        clear_btn = gr.Button("Limpar")

    upload_btn.click(
        fn=lambda x: assistant.load_and_process_pdfs(x),
        inputs=files,
        outputs=status
    )

    send_btn.click(
        fn=create_response,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot]
    )

    clear_btn.click(
        fn=lambda: [],
        inputs=[],
        outputs=chatbot
    )

# %%
try:
    demo.launch(
        share=True,
        debug=True,
        server_name="0.0.0.0"
    )
except Exception as e:
    logger.error(f"Falha ao iniciar: {str(e)}")
    raise e